# Model Evaluation 
This is the script to generate visuals and images of model results, since many of the models did not successfully train on validation sets.


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
from tensorflow.distribute import MirroredStrategy, HierarchicalCopyAllReduce

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

#tf.compat.v1.disable_eager_execution()
#tf.config.run_functions_eagerly(False)
os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"]="2"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

strategy = MirroredStrategy(cross_device_ops=HierarchicalCopyAllReduce())



INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [2]:
# validation generator
from model_utils import artificial_lunar_landscapes, generate_image_paths
from model_utils_sobel import artificial_lunar_landscapes as artificial_lunar_landscapes_sobel
from model_utils_sobel import  generate_image_paths as generate_image_paths_sobel

In [3]:
import gc

In [4]:
#
batch_size = 2
image_size = (480,720,1)
input_processed_size = (480,720,1)


In [5]:
# ORIGNIAL TRAIN TEST GENERATOR (no Sobel)
#training generator
"""
input_dir, target_dir, input_img_paths, target_img_paths = generate_image_paths(
    target_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/mask-og', 
    input_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/render-og'
)
train_generator = artificial_lunar_landscapes(
    batch_size=batch_size,
    image_size=image_size,
    input_img_paths=input_img_paths,
    target_img_paths=target_img_paths
)


#test generator
input_dir, target_dir, input_img_paths, target_img_paths = generate_image_paths(
    target_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/test/mask-og', 
    input_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/test/render-og'
)
test_generator = artificial_lunar_landscapes(
    batch_size=batch_size,
    image_size=image_size,
    input_img_paths=input_img_paths,
    target_img_paths=target_img_paths
)
"""

"\ninput_dir, target_dir, input_img_paths, target_img_paths = generate_image_paths(\n    target_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/mask-og', \n    input_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/render-og'\n)\ntrain_generator = artificial_lunar_landscapes(\n    batch_size=batch_size,\n    image_size=image_size,\n    input_img_paths=input_img_paths,\n    target_img_paths=target_img_paths\n)\n\n\n#test generator\ninput_dir, target_dir, input_img_paths, target_img_paths = generate_image_paths(\n    target_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/test/mask-og', \n    input_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/test/render-og'\n)\ntest_generator = artificial_lunar_landscapes(\n    batch_size=batch_size,\n    image_size=image_size,\n    input_img_paths=input_img_paths,\n    target_img_paths=target_img_paths\n)\n"

In [6]:
### SOBEL GENERATOR
# training generator with sobel - X output is [original_image, processed_image]
"""
input_dir, target_dir, input_img_paths, target_img_paths, process_img_paths = generate_image_paths_sobel(
    processed_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/render-processed',
    target_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/mask-og', 
    input_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/render-og'
)
train_generator_sobel = artificial_lunar_landscapes_sobel(
    batch_size=batch_size,
    image_size=image_size,
    image_processed_size = input_processed_size,
    input_img_paths=input_img_paths,
    target_img_paths=target_img_paths,
    input_img_processed_paths=process_img_paths
)

#test generator with sobel - X output is [original_image, processed_image]
input_dir, target_dir, input_img_paths, target_img_paths, process_img_paths = generate_image_paths_sobel(
    processed_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/test/render-processed',
    target_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/test/mask-og', 
    input_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/test/render-og'
)
test_generator_sobel = artificial_lunar_landscapes_sobel(
    batch_size=batch_size,
    image_size=image_size,
    image_processed_size = input_processed_size,
    input_img_paths=input_img_paths,
    target_img_paths=target_img_paths,
    input_img_processed_paths=process_img_paths
)
"""

"\ninput_dir, target_dir, input_img_paths, target_img_paths, process_img_paths = generate_image_paths_sobel(\n    processed_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/render-processed',\n    target_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/mask-og', \n    input_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/render-og'\n)\ntrain_generator_sobel = artificial_lunar_landscapes_sobel(\n    batch_size=batch_size,\n    image_size=image_size,\n    image_processed_size = input_processed_size,\n    input_img_paths=input_img_paths,\n    target_img_paths=target_img_paths,\n    input_img_processed_paths=process_img_paths\n)\n\n#test generator with sobel - X output is [original_image, processed_image]\ninput_dir, target_dir, input_img_paths, target_img_paths, process_img_paths = generate_image_paths_sobel(\n    processed_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/test/re

In [7]:
# custom functions
def weighted_bincrossentropy(true, pred, weight_zero = 0.1, weight_one = 1):
    """
    Calculates weighted binary cross entropy. The weights are fixed.
        
    This can be useful for unbalanced catagories.
    
    Adjust the weights here depending on what is required.
    
    For example if there are 10x as many positive classes as negative classes,
        if you adjust weight_zero = 1.0, weight_one = 0.1, then false positives # predict rock but actually no rock
        will be penalize 10 times as much as false negatives. # predict no rock but actually rock
    """

    # calculate the binary cross entropy
    bin_crossentropy = keras.backend.binary_crossentropy(true, pred)
    
    # apply the weights
    weights = true * weight_one + (1. - true) * weight_zero
    weighted_bin_crossentropy = weights * bin_crossentropy 

    return keras.backend.mean(weighted_bin_crossentropy)

def wbce( y_true, y_pred, weight1=1, weight0=0.1 ) :
    y_true = K.clip(y_true, K.epsilon(), 1-K.epsilon())
    y_pred = K.clip(y_pred, K.epsilon(), 1-K.epsilon())
    logloss = -(y_true * K.log(y_pred) * weight1 + (1 - y_true) * K.log(1 - y_pred) * weight0 )
    return K.mean( logloss, axis=-1)

### Functions


In [8]:
model_checkpoint_directory = '../results/simple-modeling/unet-weighted-bce-trainval/checkpoints/'
os.listdir(model_checkpoint_directory)

model_list = []
for i in range(0,20):
    model_list.append(f'model-{i}.h5')

model_list

['model-0.h5',
 'model-1.h5',
 'model-2.h5',
 'model-3.h5',
 'model-4.h5',
 'model-5.h5',
 'model-6.h5',
 'model-7.h5',
 'model-8.h5',
 'model-9.h5',
 'model-10.h5',
 'model-11.h5',
 'model-12.h5',
 'model-13.h5',
 'model-14.h5',
 'model-15.h5',
 'model-16.h5',
 'model-17.h5',
 'model-18.h5',
 'model-19.h5']

In [9]:
def model_func(model_directory=None, custom_objects=None):
    # LOAD GENERATOR#
    input_dir, target_dir, input_img_paths, target_img_paths, process_img_paths = generate_image_paths_sobel(
        processed_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/render-processed',
        target_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/mask-og', 
        input_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/render-og'
    )
    train_generator = artificial_lunar_landscapes_sobel(
        batch_size=batch_size,
        image_size=image_size,
        image_processed_size = input_processed_size,
        input_img_paths=input_img_paths,
        target_img_paths=target_img_paths,
        input_img_processed_paths=process_img_paths
    )

    #test generator with sobel - X output is [original_image, processed_image]
    input_dir, target_dir, input_img_paths, target_img_paths, process_img_paths = generate_image_paths_sobel(
        processed_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/test/render-processed',
        target_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/test/mask-og', 
        input_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/test/render-og'
    )
    test_generator = artificial_lunar_landscapes_sobel(
        batch_size=batch_size,
        image_size=image_size,
        image_processed_size = input_processed_size,
        input_img_paths=input_img_paths,
        target_img_paths=target_img_paths,
        input_img_processed_paths=process_img_paths
    )


    #END GENERATOR #
    
    
    model = load_model(model_directory, custom_objects=custom_objects)
    print(model.summary())
    
    #with strategy.scope():
    print("Evaluating on Training Set")
    results_train = model.evaluate(train_generator)
    
    print("Evaluating on Test Set")
    results_val = model.evaluate(test_generator)
    
    del test_generator
    del train_generator
    del model
    K.clear_session()
    gc.collect()
    keras.backend.clear_session()
    for i in range(3): gc.collect()
    
    
    return results_train, results_val




def model_func_branches_training_eval(model_directory, custom_objects):
    input_dir, target_dir, input_img_paths, target_img_paths, process_img_paths = generate_image_paths_sobel(
        processed_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/render-processed',
        target_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/mask-og', 
        input_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/train/render-og'
    )
    train_generator_sobel = artificial_lunar_landscapes_sobel(
        batch_size=batch_size,
        image_size=image_size,
        image_processed_size = input_processed_size,
        input_img_paths=input_img_paths,
        target_img_paths=target_img_paths,
        input_img_processed_paths=process_img_paths
    )
    print("Evaluating on Training Set")
    model = load_model(model_directory, custom_objects=custom_objects)
    print(model.summary())
    results_train = model.evaluate(train_generator_sobel)
    del model
    K.clear_session()
    gc.collect()
    keras.backend.clear_session()
    for i in range(3): gc.collect()
    return results_train

def model_func_branches_testing_eval(model_directory, custom_objects):
    input_dir, target_dir, input_img_paths, target_img_paths, process_img_paths = generate_image_paths_sobel(
        processed_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/test/render-processed',
        target_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/test/mask-og', 
        input_dir='D:/TJPersonalCloud/Programming/msds-thesis-data/data-ml/split-data/test/render-og'
    )
    test_generator_sobel = artificial_lunar_landscapes_sobel(
        batch_size=batch_size,
        image_size=image_size,
        image_processed_size = input_processed_size,
        input_img_paths=input_img_paths,
        target_img_paths=target_img_paths,
        input_img_processed_paths=process_img_paths
    )
    model = load_model(model_directory, custom_objects=custom_objects)
    results_val = model.evaluate(test_generator_sobel)
    del model
    K.clear_session()
    gc.collect()
    keras.backend.clear_session()
    for i in range(3): gc.collect()
    return results_val

def model_func_branches(model_directory=None, custom_objects=None):
    """ LOAD GENERATOR"""
    #model = load_model(model_directory, custom_objects=custom_objects)
    results_train = model_func_branches_training_eval(model_directory, custom_objects)
    results_val = model_func_branches_testing_eval(model_directory, custom_objects)

    #test generator with sobel - X output is [original_image, processed_image]


    """ END GENERATOR """
    
    
    
    
    
    #with strategy.scope():
    #del train_generator_sobel
    print("Evaluating on Test Set")
    
    
    #del test_generator_sobel
    

    
    
    return results_train, results_val

In [10]:
def model_evaluation_run(model_checkpoint_directory, model, custom_objects):
    # run evaluation of models
    temp_train_results, temp_val_results = model_func(model_checkpoint_directory+model,custom_objects=custom_objects)
    
    # create dataframe of particular model results
    temp_train_results_df = pd.DataFrame(temp_train_results).T
    temp_val_results_df = pd.DataFrame(temp_val_results).T
    
    # name the columns to match metrics
    temp_train_results_df.columns = ['loss','accuracy','mean_iou','recall','precision']
    temp_val_results_df.columns = ['loss','accuracy','mean_iou','recall','precision']
    # name index to match model name
    temp_train_results_df.index = [model]
    temp_val_results_df.index = [model]

    return temp_train_results_df, temp_val_results_df

def model_evaluation_run_branches(model_checkpoint_directory, model, custom_objects):
    # run evaluation of models
    temp_train_results, temp_val_results = model_func_branches(model_checkpoint_directory+model,custom_objects=custom_objects)
    
    # create dataframe of particular model results
    temp_train_results_df = pd.DataFrame(temp_train_results).T
    temp_val_results_df = pd.DataFrame(temp_val_results).T
    
    # name the columns to match metrics
    temp_train_results_df.columns = ['loss','accuracy','mean_iou','recall','precision']
    temp_val_results_df.columns = ['loss','accuracy','mean_iou','recall','precision']
    # name index to match model name
    temp_train_results_df.index = [model]
    temp_val_results_df.index = [model]

    return temp_train_results_df, temp_val_results_df

# Model Evaluation Start

In [11]:
from tensorflow.compat.v1.keras.backend import set_session, clear_session, get_session
from tensorflow.compat.v1 import ConfigProto, Session
#import tensorflow

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(Session(config=config))

In [12]:
reset_keras()

0


## y-model-branches

In [13]:
"""
# testing structure
#directory of models to iterate through
model_checkpoint_directory = 'D:/ProgrammingD/github/thesis/modeling/results/custom-modeling/y-model-branches/checkpoints/traintestrun/'
model_list = os.listdir(model_checkpoint_directory)
print(model_list)

#with strategy.scope():

result_train_df = pd.DataFrame(columns=['loss','accuracy','mean_iou','recall','precision'])
result_val_df = pd.DataFrame(columns=['loss','accuracy','mean_iou','recall','precision'])
for model in model_list:
    print("MODEL TO BE EVALUATED: ")
    print(model)
    temp_train_results_df, temp_val_results_df = model_evaluation_run(model_checkpoint_directory, model, custom_objects={'weighted_bincrossentropy': weighted_bincrossentropy})

    result_train_df = result_train_df.append(temp_train_results_df,ignore_index=True)
    result_val_df = result_val_df.append(temp_val_results_df, ignore_index=True)

    print("Current Training Results")
    print("------------")
    print(result_train_df)
    print("Current Validation Results")
    print("------------")
    print(result_val_df)
    print("...Evaluation Complete")

    result_train_df.to_csv('training_results_y_model_branches.csv')
    result_val_df.to_csv('validation_results_unet_y_model_branches.csv')

    K.clear_session()
    gc.collect()
    keras.backend.clear_session()
    for i in range(3): gc.collect()
    reset_keras()
    """

'\n# testing structure\n#directory of models to iterate through\nmodel_checkpoint_directory = \'D:/ProgrammingD/github/thesis/modeling/results/custom-modeling/y-model-branches/checkpoints/traintestrun/\'\nmodel_list = os.listdir(model_checkpoint_directory)\nprint(model_list)\n\n#with strategy.scope():\n\nresult_train_df = pd.DataFrame(columns=[\'loss\',\'accuracy\',\'mean_iou\',\'recall\',\'precision\'])\nresult_val_df = pd.DataFrame(columns=[\'loss\',\'accuracy\',\'mean_iou\',\'recall\',\'precision\'])\nfor model in model_list:\n    print("MODEL TO BE EVALUATED: ")\n    print(model)\n    temp_train_results_df, temp_val_results_df = model_evaluation_run(model_checkpoint_directory, model, custom_objects={\'weighted_bincrossentropy\': weighted_bincrossentropy})\n\n    result_train_df = result_train_df.append(temp_train_results_df,ignore_index=True)\n    result_val_df = result_val_df.append(temp_val_results_df, ignore_index=True)\n\n    print("Current Training Results")\n    print("------

### y model branches filtersample

In [14]:
# testing structure
#directory of models to iterate through
model_checkpoint_directory = 'D:/ProgrammingD/github/thesis/modeling/results/custom-modeling/y-model-branches-filtersample/checkpoints/traintestrun/'
model_list = sorted(os.listdir(model_checkpoint_directory))
print(model_list)

#with strategy.scope():

result_train_df = pd.DataFrame(columns=['loss','accuracy','mean_iou','recall','precision'])
result_val_df = pd.DataFrame(columns=['loss','accuracy','mean_iou','recall','precision'])
for model in model_list[17:]:
    print("MODEL TO BE EVALUATED: ")
    print(model)
    temp_train_results_df, temp_val_results_df = model_evaluation_run(model_checkpoint_directory, model, custom_objects={'weighted_bincrossentropy': weighted_bincrossentropy})

    result_train_df = result_train_df.append(temp_train_results_df,ignore_index=True)
    result_val_df = result_val_df.append(temp_val_results_df, ignore_index=True)

    print("Current Training Results")
    print("------------")
    print(result_train_df)
    print("Current Validation Results")
    print("------------")
    print(result_val_df)
    print("...Evaluation Complete")

    result_train_df.to_csv('training_results_y_model_branches_filtersample.csv')
    result_val_df.to_csv('validation_results_unet_y_model_branches_filtersample.csv')

    K.clear_session()
    gc.collect()
    keras.backend.clear_session()
    for i in range(3): gc.collect()
    reset_keras()

['model.01-0.09-0.73.h5', 'model.02-0.07-0.82.h5', 'model.03-0.07-0.85.h5', 'model.04-0.06-0.85.h5', 'model.05-0.06-0.85.h5', 'model.06-0.06-0.86.h5', 'model.07-0.06-0.87.h5', 'model.08-0.06-0.87.h5', 'model.09-0.06-0.88.h5', 'model.10.h5', 'model.11.h5', 'model.12.h5', 'model.13.h5', 'model.14.h5', 'model.15.h5', 'model.16.h5', 'model.17.h5', 'model.18.h5', 'model.19.h5', 'model.20.h5']
MODEL TO BE EVALUATED: 
model.18.h5
Number of images:  7813
Number of processed:  7813
Number of targets:  7813
Number of images:  1953
Number of processed:  1953
Number of targets:  1953
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 480, 720, 1) 0                                            
__________________________________________________________________________________________________
input_1 (I